# Using DESeq2 for sample normalization and statistical analysis to identify mRNA associated with smoke exposure

Code includes: data organization,PCA, surrogate variable analysis, and other steps prior to DESeq2

In [11]:
# Installing DESeq2:
# Note that BiocManager allows you to access the Bioconductor repo in which DeSeq2 package is. DeSeq2 pacakge has a "tibble" "RcppArmadillo" and "rlang" dependency.
# to install RcppArmadillo, you must install a Fortan compiler (I used gfortran-6.1.pkg, https://cran.r-project.org/bin/macosx/tools/).

if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

install.packages("tibble")
install.packages("rlang")
install.packages("RcppArmadillo")

# Install DESeq and SVA:
BiocManager::install("DESeq2", version = "3.11")
BiocManager::install("sva", version ="3.11")
BiocManager::install("BiocManager")

# Install plot packages
install.packages("ggbeeswarm") # note that I had to manually install this package using tools --> install packages
#install.packages("gridExtra")

# Install tidyverse packages 
#install.packages("tidyverse")

# Install gplots packages
install.packages("gplots")

# Install RColorBrewer package
install.packages("RColorBrewer")

# Install scales (needed for tidyverse activation):
install.packages("scales")

# Install data.tables
install.packages("data.table")

Warning message in install.packages("tibble"):
“installation of package ‘tibble’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in install.packages("rlang"):
“installation of package ‘rlang’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in install.packages("RcppArmadillo"):
“installation of package ‘RcppArmadillo’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Bioconductor version 3.11 (BiocManager 1.30.10), R 4.0.2 (2020-06-22)

Installing package(s) 'DESeq2'

also installing the dependencies ‘bit’, ‘bitops’, ‘bit64’, ‘plogr’, ‘RCurl’, ‘GenomeInfoDbData’, ‘zlibbioc’, ‘lambda.r’, ‘futile.options’, ‘RSQLite’, ‘XML’, ‘GenomeInfoDb’, ‘XVector’, ‘DelayedArray’, ‘futile.logger’, ‘snow’, ‘BH’, ‘AnnotationDbi’, ‘annotate’, ‘S4Vectors’, ‘IRanges’, ‘GenomicRanges’, ‘SummarizedExperimen

In [ ]:
# Activating appropriate libraries (skip step above if everything is already installed)
library(data.table)
library(rlang)
library(DESeq2)
library(limma)
library(sva)
library(ggplot2)
library(stats)
library(scales)
library(ggbeeswarm)
library(gridExtra)
library(tidyverse)
library(dplyr)
library(gplots)
library(RColorBrewer)
library(data.table)
library(readxl)

In [12]:
# Set working directory
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/2_Wildfire_Analysis/Input')

# Create an output folder (make sure to make the folder first, and then point to it here)
Output <- ("/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/2_Wildfire_Analysis/Output")

Loading count data and metadata (subject information) & organizing and filtering to keep subjects present in both files

In [14]:
library(tidyverse) #remove

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.3     ✔ purrr   0.3.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%@%()         masks rlang::%@%()
✖ rlang:::=()          masks data.table:::=()
✖ purrr::as_function() masks rlang::as_function()
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ purrr::flatten()     masks rlang::flatten()
✖ purrr::flatten_chr() masks rlang::flatten_chr()
✖ purrr::flatten_dbl() masks rlang::flatten_dbl()
✖ purrr::flatten_int() masks rlang::flatten_int()
✖ purrr::flatten_lgl() masks rlang::flatten_lgl()
✖ purrr::flatten_raw() masks rlang::flatten_raw()
✖ purrr::invoke()      masks rlang::invoke()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ purrr::list_

In [19]:
library(readxl) #remove

In [49]:
# Read in count data
# countdata <- read.table(file = 'Wildfire_GeneCounts_102320.csv', header = TRUE)
countdata <- read_csv(file = 'Wildfire_GeneCounts_102320.csv') 

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_double(),
  X1 = col_character()
)

See spec(...) for full column specifications.



In [43]:
countdata = countdata%>%
    filter(colnames(countdata)[1:5] %in% c('MH112', 'MH113', 'MH114', 'MH115', 'MH116', 'MH117', 'MH118','MH101',
                                           'MH102', 'MH103', 'MH104', 'MH105','MH106'))

ERROR: Error: Problem with `filter()` input `..1`.
[31m✖[39m Input `..1` must be of size 30146 or 1, not size 5.
[34mℹ[39m Input `..1` is ``%in%`(...)`.


In [45]:
colnames(countdata[1:10])

[1] "X1"                    "M1_RedOakSmolder"      "M10_PeatSmolder"      
 [4] "M101_Saline"           "M107_RedOakFlame"      "M11_PeatSmolder"      
 [7] "M113_PeatFlame"        "M119_PineNeedlesFlame" "M12_PeatSmolder"      
[10] "M125_PineFlame"

In [46]:
colnames(countdata[1:10])[1:5]

[1] "X1"               "M1_RedOakSmolder" "M10_PeatSmolder"  "M101_Saline"     
[5] "M107_RedOakFlame"

In [70]:
#only interested in comparing mouse heart peat flame (MH112-MH118) vs. mouse heart saline samples (MH101-MH106)
for (i in 1:length(colnames(countdata))){
#     if(i < 10){
#         print(i)
#         print(colnames(countdata)[i])
#     }
    colname = colnames(countdata)[i]
    for (j in 1:length(colname)){
        if (substr(colname,j,j+1) == 'MH'){ #saline
            print(colname)
            print(j)
            #mouse heart peat flame
#             if (substr(as.numeric(colname), j+3,j+6) <= 118){ #| substr(as.numeric(colname), j,j+3) >= 112){ 
#                 print(colname)
#             }
            
        }
    }
}

ERROR: Error in substr(colname): argument "start" is missing, with no default


In [67]:
a = colnames(countdata[2])
substr(a,1,4)

[1] "M1_R"

In [59]:
string = 'hershey'
substr(string,1,3)

[1] "her"

In [75]:
for (i in 1:length(strsplit(string))){
    print(i)
    print(string[i])
}

ERROR: Error in strsplit(string): argument "split" is missing, with no default


In [22]:
dim(countdata) #171 samples (plus Gene ID column), 30146 genes
# visualize this data quickly by viewing top left corner:
countdata[1:3,1:6]

# Check for duplicate gene IDs in the countdata 
Dups <- duplicated(countdata[,1])
summary(Dups)
# Not an issue for this wildfire dataset

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_double(),
  X1 = col_character()
)

See spec(...) for full column specifications.



[1] 30146   171

X1,M1_RedOakSmolder,M10_PeatSmolder,M101_Saline,M107_RedOakFlame,M11_PeatSmolder
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Bcat1_29039,0,0,0,0,0
Laptm5_29040,0,2,2,1,4
Skp2_29041,0,0,0,0,0


   Mode   FALSE 
logical   30146 

In [39]:
colnames(countdata)

[1] "Gene"                   "M100_EucalyptusSmolder" "M101_Saline"           
  [4] "M102_Saline"            "M103_Saline"            "M104_Saline"           
  [7] "M105_Saline"            "M106_Saline"            "M107_RedOakFlame"      
 [10] "M108_RedOakFlame"       "M109_RedOakFlame"       "M10_PeatSmolder"       
 [13] "M110_RedOakFlame"       "M111_RedOakFlame"       "M112_RedOakFlame"      
 [16] "M113_PeatFlame"         "M114_PeatFlame"         "M115_PeatFlame"        
 [19] "M116_PeatFlame"         "M117_PeatFlame"         "M118_PeatFlame"        
 [22] "M119_PineNeedlesFlame"  "M11_PeatSmolder"        "M120_PineNeedlesFlame" 
 [25] "M121_PineNeedlesFlame"  "M122_PineNeedlesFlame"  "M123_PineNeedlesFlame" 
 [28] "M124_PineNeedlesFlame"  "M125_PineFlame"         "M126_PineFlame"        
 [31] "M127_PineFlame"         "M128_PineFlame"         "M129_PineFlame"        
 [34] "M12_PeatSmolder"        "M130_PineFlame"         "M131_PineFlame"        
 [37] "M132_PineFlame"         "M133_PineFlame"         "M134_PineFlame"        
 [40] "M135_PineFlame"         "M136_PineFlame"         "M137_LPS"              
 [43] "M138_LPS"               "M139_LPS"               "M13_PeatSmolder"       
 [46] "M140_LPS"               "M14_PeatSmolder"        "M15_PeatSmolder"       
 [49] "M16_PeatSmolder"        "M17_PeatSmolder"        "M18_PeatSmolder"       
 [52] "M19_PineSmolder"        "M1_RedOakSmolder"       "M20_PineSmolder"       
 [55] "M21_PineSmolder"        "M22_PineSmolder"        "M23_PineSmolder"       
 [58] "M24_PineSmolder"        "M25_EucalyptusSmolder"  "M26_EucalyptusSmolder" 
 [61] "M27_EucalyptusSmolder"  "M28_EucalyptusSmolder"  "M29_EucalyptusSmolder" 
 [64] "M2_RedOakSmolder"       "M30_EucalyptusSmolder"  "M31_Saline"            
 [67] "M32_Saline"             "M33_Saline"             "M34_Saline"            
 [70] "M35_Saline"             "M36_Saline"             "M37_RedOakFlame"       
 [73] "M38_RedOakFlame"        "M39_RedOakFlame"        "M3_RedOakSmolder"      
 [76] "M40_RedOakFlame"        "M41_RedOakFlame"        "M42_RedOakFlame"       
 [79] "M43_PeatFlame"          "M44_PeatFlame"          "M45_PeatFlame"         
 [82] "M46_PeatFlame"          "M47_PeatFlame"          "M48_PeatFlame"         
 [85] "M49_PineNeedlesFlame"   "M4_RedOakSmolder"       "M50_PineNeedlesFlame"  
 [88] "M51_PineNeedlesFlame"   "M52_PineNeedlesFlame"   "M53_PineNeedlesFlame"  
 [91] "M54_PineNeedlesFlame"   "M55_PineFlame"          "M56_PineFlame"         
 [94] "M57_PineFlame"          "M58_PineFlame"          "M59_PineFlame"         
 [97] "M5_RedOakSmolder"       "M60_PineFlame"          "M61_EucalyptusFlame"   
[100] "M62_EucalyptusFlame"    "M63_EucalyptusFlame"    "M64_EucalyptusFlame"   
[103] "M65_EucalyptusFlame"    "M66_EucalyptusFlame"    "M67_LPS"               
[106] "M68_LPS"                "M69_LPS"                "M6_RedOakSmolder"      
[109] "M70_LPS"                "M71_RedOakSmolder"      "M72_RedOakSmolder"     
[112] "M73_RedOakSmolder"      "M74_RedOakSmolder"      "M75_RedOakSmolder"     
[115] "M76_RedOakSmolder"      "M77_PeatSmolder"        "M78_PeatSmolder"       
[118] "M79_PeatSmolder"        "M7_PeatSmolder"         "M80_PeatSmolder"       
[121] "M81_PeatSmolder"        "M82_PeatSmolder"        "M83_PineNeedlesSmolder"
[124] "M84_PineNeedlesSmolder" "M85_PineNeedlesSmolder" "M86_PineNeedlesSmolder"
[127] "M87_PineNeedlesSmolder" "M88_PineNeedlesSmolder" "M89_PineSmolder"       
[130] "M8_PeatSmolder"         "M90_PineSmolder"        "M91_PineSmolder"       
[133] "M92_PineSmolder"        "M93_PineSmolder"        "M94_PineSmolder"       
[136] "M95_EucalyptusSmolder"  "M96_EucalyptusSmolder"  "M97_EucalyptusSmolder" 
[139] "M98_EucalyptusSmolder"  "M99_EucalyptusSmolder"  "M9_PeatSmolder"        
[142] "MH101_Saline"           "MH102_Saline"           "MH103_Saline"          
[145] "MH104_Saline"           "MH105_Saline"           "MH106_Saline"          
[148] "MH107_RedOakFlame"      "M

In [21]:
# Read in metadata (subject info file)
subjectinfo <- read_excel("SampleID_Information_102320.xlsx")
dim(subjectinfo) #170 rows, 7 col

# Visualize this data quickly by viewing top left corner:
subjectinfo[1:3,1:6]

[1] 170   7

SampleID_BioSpyderCountFile,PlateBatch,MouseID,MouseID_CAPS,FullTreatment,FullID
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Plate1-m1-RNA,Plate1,m1,M1,RedOakSmolder,M1_RedOakSmolder
Plate1-m10-RNA,Plate1,m10,M10,PeatSmolder,M10_PeatSmolder
Plate1-m101-RNA,Plate1,m101,M101,Saline,M101_Saline


Applying first round of various filters to these imported data files, including:
- (1) First to first keep subjects present in both files
- (2) Background filter, to filter out genes that were universally lowly expressed
- (3) Check subject filter, to check that we have filtered out subjects that had samples present all values of zero (and thus not detected)
- (4) Re-filter to include subjects meeting the above filters in (2) and (3)

In [24]:
##########
##########
# (1) Filtering to first keep subjects present in both files
##########
##########

# Pull all mRNAseq IDs that overlap between countdata & subjectinfo
keep_samples <- colnames(countdata[, colnames(countdata) %in% subjectinfo$FullID]) 
# Note that in this case, this represents 170 IDs

In [27]:
# Filter the countdata dataframe for these IDs
# Because the first column is the gene identifier information, this will get lost if we don't also grab that 
# separately (hence, a few steps here)
countdata_temp <- countdata[, colnames(countdata) %in% keep_samples] # pulling countdata, without gene identifier col
countdata <- cbind(countdata[,1], countdata_temp)    # adding the gene identifier column back in
colnames(countdata)[1] <- "Gene"   # renaming the gene identifer column
countdata_temp <- NULL    # removing temporary dataframe

# Viewing merged, filtered count data
countdata[1:3,1:10]

# Filter the subjectinfo dataframe for these IDs
subjectinfo <- subjectinfo[subjectinfo$FullID %in% keep_samples, ]

,Gene,M1_RedOakSmolder,M10_PeatSmolder,M101_Saline,M107_RedOakFlame,M11_PeatSmolder,M113_PeatFlame,M119_PineNeedlesFlame,M12_PeatSmolder,M125_PineFlame
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Bcat1_29039,0,0,0,0,0,0,0,0,0
2,Laptm5_29040,0,2,2,1,4,22,3,1,4
3,Skp2_29041,0,0,0,0,0,0,0,0,0


In [31]:
##########
##########
# (2) Background filter, to filter out genes that were universally lowly expressed
##########
##########


# Create variable for number of samples, assuming that one variable is Gene: 
numberofsamples = (ncol(countdata)-1)

# Create a variable that is the median of all expression levels across all samples and genes
totalmedian = median(as.matrix(countdata[,2:ncol(countdata)]))

# Create a list of genes that pass the background filter:
Genes_backgroundfiltered <- countdata %>% 
  #gather so that there is one row for each gene-sample pairing
  gather(key = "sampleID", value = "expression", 2:ncol(countdata)) %>% 
  #add a variable that takes the value 1 is the expression value is above the median 
  mutate(abovemedian = ifelse(expression > totalmedian,1,0)) %>% 
  #sum the abovemedian variable created above by gene so that totalabovemedian calcualtes the number of samples for each gene
  #that have expression levels above the median
  group_by(Gene) %>% 
  summarize(totalabovemedian = sum(abovemedian)) %>% 
  #remove all genes that have a totalabovemedian equal to less than a specified percentage of the total number of samples 
  filter(totalabovemedian > (0.20*numberofsamples)) %>% 
  #select only gene so that you essentially have a list of the genes to include 
  select(Gene) 
# Select genes that passed background filter only to proceed with analysis
countdata <- left_join(Genes_backgroundfiltered, countdata, by = "Gene")
# This resulted in filtering the original list of 30,146 genes down to 17,356 genes

`summarise()` ungrouping output (override with `.groups` argument)



In [36]:
##########
##########
#### (3) Check subject filter, to check that we have filtered out subjects that had samples present all values of zero
#### (and thus not detected)
##########
##########

# Note that this code was originally drafted using a transposed version of the countdata dataframe
# So here, rather than modifying, we will transpose and then transpose back at the end

#transpose count data 
countdata_t <- countdata %>% 
  gather(key = "sampleID", value = "expression", 2:ncol(countdata)) %>% 
  spread(Gene, expression)  
# view transposed count data
countdata_t[1:4,1:10]

countdata_t <- countdata_t %>% 
  #create variable that is the sum of count values across all genes, for each sample
  mutate(rowsum =  rowSums(countdata_t[,2:nrow(countdata_t)],na.rm  =  FALSE, dims = 1L)) 

#identify the samples that will be removed, for records 
samples_zerocounts <- countdata_t %>% 
  filter(rowsum  == 0) %>% 
  select("sampleID") 

countdata_t <- countdata_t %>% 
  #remove samples that have a total sum of all counts of zero 
  filter(rowsum != 0) %>% 
  select(-rowsum)

# SAMPLES REMOVED: No samples were removed 


# Transposing the countdata back such that the genes are the rownames and the samples are the columnnames
countdata <- countdata_t %>% 
  gather(Gene, value, 2:ncol(countdata_t)) %>% 
  spread(sampleID, value) 

In [ ]:
##########
##########
# (4) Re-filter to include subjects meeting the above filters in (2) and (3)
##########
##########

# Pull all mRNAseq IDs that overlap between countdata & subjectinfo
keep_samples <- colnames(countdata[, colnames(countdata) %in% subjectinfo$FullID]) 
# Note that in this case, this represents 372 IDs

# Filter the countdata dataframe for these IDs
# Because the first column is the gene identifier information, this will get lost if we don't also grab that separately (hence, a few steps here)
countdata_temp <- countdata[, colnames(countdata) %in% keep_samples]   # pulling all countdata, without gene identifier column
countdata <- cbind(countdata$Gene, countdata_temp)    # adding the gene identifier column back in
colnames(countdata)[1] <- "Gene"   # renaming the gene identifer column
# Viewing merged, filtered count data
countdata[1:3,1:10]

# Filter the subjectinfo dataframe for these IDs
subjectinfo <- subjectinfo[subjectinfo$mRNAseq_ID %in% keep_samples, ]

Create dataframes that are formatted for proceeding code, as well as DESeq2 functions countdata and coldata

In [ ]:
# Make the gene variable the rowname 
countdata <- countdata %>% 
  column_to_rownames(var="Gene") 

# Creating the coldata object, based on information in the subjectinfo file
coldata <- subjectinfo
# Let's change the name of the primary ID column, to use from here on out
names(coldata)[names(coldata) == 'mRNAseq_ID'] <- 'ID'


# Set the rownames of coldata and column names of countdata to be in the same order 
countdata <- setcolorder(countdata, as.character(coldata$ID))

# Double checking that the same variables appear between the two dataframes
setequal(as.character(coldata$ID), colnames(countdata))

# Additionally checking that not only the sets of variables are the same, but that they are in the same order
identical(as.character(coldata$ID), colnames(countdata))

RNASeq QA/QC on raw count data to identify potential outlier samples. (This may or may not result in another filter step.)

In [ ]:
# One way to evaluate sequencing data quality / identify potential outliers is through Principal Component Analysis (PCA):
# PCA helps in identifying outlying samples for quality control, and gives a feeling for the principal causes of variation in a dataset

# Calculate principal components using transposed count data
pca <- prcomp(t(countdata))
screeplot(pca)
#this scree plot indicates that nearly all variation is explained in PC1,2 and 3, 
#thus PCA is a useful exercise 

# Make dataframe for PCA plot generation using first three components
pca_df <- data.frame(PC1 = pca$x[,1], PC2 = pca$x[,2], PC3 = pca$x[,3], Sample=colnames(countdata))
# Add attributes (covariates from coldata) to pca df
pca_df <- merge(pca_df, coldata, by.x="Sample", by.y="ID")



# Calculating percent of the variation that is captured by each principal component
pca_percent <- round(100*pca$sdev^2/sum(pca$sdev^2),1)

# Generating PCA plot annotated by sex 
ggplot(pca_df, aes(PC1,PC2, color = sex))+
  geom_point(size=6) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

# Generating PCA plot annotated by maternal age 
ggplot(pca_df, aes(PC1,PC2, color = mage))+
  geom_point(size=6) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

# Generating PCA plot annotated by gestational age 
ggplot(pca_df, aes(PC1,PC2, color = gadays))+
  geom_point(size=6) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

# Generating PCA plot annotated by race 
ggplot(pca_df, aes(PC1,PC2, color = race1))+
  geom_point(size=6) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

# Lets identify the outliers
ggplot(pca_df, aes(PC1,PC2))+
  geom_text(aes(label=Sample, size=3)) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))


# This would suggest that LS18017 and LS18097 are potential outliers, as well as potentially LS18183 and LS18076
# We will conduct heirachical clustering to investigate further and to determine if we should also remove these samples


# create a dataframe with samples as rows and genes as columns to input into hclust
countdata_forclustering <- t(countdata)
countdata_forclustering[1:5,1:10]


# run heirarchical clustering
hc <-hclust(dist(countdata_forclustering))
jpeg(filename= paste0(Output_Folder,"/",cur_date, "_heirachicalclusteringforoutliers.jpeg"),width = 4500, height = 350)
plot(hc)
dev.off()

# Here, the four aforementioned samples were clearly separate from the rest, so should be removed

Removing sample(s) with potential QA/QC issues & Rerunning PCA analysis with outlier removed

In [ ]:
# Remove these subject IDs from our "keep_samples" vector
keep_samples <- keep_samples[! keep_samples %in% c("LS18017", "LS18097", "LS18183", "LS18076")]
# Note that in this case, this represents 368 IDs

# Filter the countdata dataframe for these IDs
countdata <- countdata[, colnames(countdata) %in% keep_samples]
countdata[1:5, 1:10]

# Filter the coldata dataframe for these IDs
coldata <- coldata[coldata$ID %in% keep_samples, ]



# Export the raw data for just the included samples to potentially generate plots outside of R
write.csv(countdata, paste0(Output_Folder,"/", cur_date, "_RawCounts_SubjectsIncludedinModel.csv"), row.names= TRUE)
write.csv(coldata, paste0(Output_Folder,"/", cur_date, "_SubjectInfo_SubjectsIncludedinModel.csv"), row.names= FALSE)
# Note that if, for future studies, we exclude subjects with missing covariate data, that step will have to incorporated prior to this export
# But for this analysis, we're imputing missing covariate data, so went ahead and exported raw data here



# Calculate principal components using transposed count data
pca <- prcomp(t(countdata))
screeplot(pca)
#this scree plot indicates that nearly all variation is explained in PC1,2 and 3, 
#thus PCA is a useful exercise 

# Make dataframe for PCA plot generation using first three components
pca_df <- data.frame(PC1 = pca$x[,1], PC2 = pca$x[,2], PC3 = pca$x[,3], Sample=colnames(countdata))
# Add attributes (covariates from coldata) to pca df
pca_df <- merge(pca_df, coldata, by.x="Sample", by.y="ID")

# Calculating percent of the variation that is captured by each principal component
pca_percent <- round(100*pca$sdev^2/sum(pca$sdev^2),1)

# Generating PCA plot annotated by sex 
ggplot(pca_df, aes(PC1,PC2, color = sex))+
  geom_point(size=6) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

# Generating PCA plot annotated by maternal age 
ggplot(pca_df, aes(PC1,PC2, color = mage))+
  geom_point(size=6) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

# Generating PCA plot annotated by gestational age 
ggplot(pca_df, aes(PC1,PC2, color = gadays))+
  geom_point(size=6) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

# Generating PCA plot annotated by race 
ggplot(pca_df, aes(PC1,PC2, color = race1))+
  geom_point(size=6) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

# Lets identify potential outliers (although here, since this is the 2nd run, we would be much stricter on this; meaning that there would have to be a super obvious outlier, or else the cycle would just continue)
ggplot(pca_df, aes(PC1,PC2))+
  geom_text(aes(label=Sample, size=3)) +
  labs(x=paste0("PC1 (",pca_percent[1],"%)"), y=paste0("PC2 (",pca_percent[2],"%)"))

Setting up the actual DESeq2 function and associated algorithm ("experiment"). More information can be found about the DeSeq2 package [here](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0550-8)

In [ ]:
# At this point, we need to make sure that the coldata is in the following format:
# rownames = the subject ID that matches the countdata
# all other columns indicate the covariate data that we may/may not include in the final experiment
coldata[1:5, 1:7]   # viewing data


# At this point, we need to make sure that the countdata is in the following format:
# rownames = the gene identifiers
# all other columns indicate individual samples that match (and are in the same order as) the coldata
countdata[1:5, 1:10]   # viewing data


# all values should be integers (DESeq2 requires integer count values), need to convert values here if haven't already
sapply(countdata, class)   # checking the class of the values
countdata_2 <- as.data.frame(sapply(countdata, as.integer))   # converting into integers if showing numeric
sapply(countdata_2, class)   # re-checking the class of the values
row.names(countdata_2) <- row.names(countdata)  # this removed the row names (gene identifiers), so had to merge these back
countdata <- countdata_2



# Create the final DESeq2 experiment, with appropriate experimental design:
# Note in the design: last factor indicates the factor we want to test the effect of; the first factor(s) = factors we want to control for
# Note that in this particular experiment, it gives us a warning that we are using integer data as numeric variables - this is what we want for some variables

dds = DESeqDataSetFromMatrix(countData = countdata,
                             colData = coldata,
                             design = ~race1 + mage + sex + gadays + zcat4 + medu + ASD)

# View what the experiment contains
dds


# Let's also make sure that we have the main contrast (last term) in the order we want to calculate appropriate fold change values
# In this case, we want ASD=0 as denominators (first level, aka, the "control" level), and ASD=1, cases as numerators
dds$ASD <- relevel (dds$ASD, "0")



# Need to next, estimate the size factors, since size factors are used to normalize the counts (next step)
# The "iterate" estimator iterates between estimating the dispersion with a design of ~1, and finding a size factor vector by numerically optimizing the likelihood of the ~1 model.
dds <- estimateSizeFactors(dds)
sizeFactors(dds) #check size factors

#################################################################################################
#################################################################################################
#### Pulling and exporting normalized count, for internal records and for potential future plots/figures
#################################################################################################
#################################################################################################
# Note that code is drafted to output several forms of normalized data - so users can choose which applies to their external purposes the best
# Also note that variance stabilized (vsd) values are commonly used for figures, which requires the experiment to run and adjust for covariates below, so will export these in the code below


# normalized counts:
normcounts<- counts(dds, normalized=TRUE)
write.csv(normcounts, paste0(Output_Folder,"/", cur_date, "_NormCounts_IncludedSubjects.csv"), row.names=TRUE)

# pseudocounts to make plots easier:
ps_normcounts <- normcounts + 1
write.csv(ps_normcounts, paste0(Output_Folder,"/",cur_date, "_NormCounts_ps_IncludedSubjects.csv"),row.names=TRUE)

# log2 pseudocounts (y=log2(n+1))
log2normcounts <- log2(normcounts+1)
write.csv(log2normcounts, paste0(Output_Folder,"/", cur_date, "_NormCounts_pslog2_IncludedSubjects.csv"), row.names=TRUE)




#################################################################################################
#################################################################################################
#### Surrogate variable analysis (SVA) to capture potential variances between samples (including batch, but also sample / cell type heterogeneity)
#################################################################################################
#################################################################################################
## For more information: https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.0030161
## "Capturing Heterogeneity in Gene Expression Studies by Surrogate Variable Analysis", Jeffrey T Leek,  John D Storey


# Set the model matrix for what's being used to fit the data (minus the SVA variables)
mod <- model.matrix(~race1 + mage + sex + gadays + zcat4 + medu + ASD, colData(dds))


# Set the null model matrix being compared against when fitting the data for the SVA analysis
mod0 <- model.matrix( ~1, colData(dds))


# Running SVA, here, using number of SVs = 3
# Note that this number of significant SVs selection was based on several iterations, where different numbers of SVs were tested for the ELGAN dataset
svseq <- svaseq(normcounts, mod, mod0, n.sv=3)

# I also tested an iteration where svaseq calculated the number of significant SVs, using the following code:
# svseq <- svaseq(normcounts, mod, mod0, n.sv=NULL)  
# Number of significant surrogate variables is much higher, but this muted global signal responses


#################
#################
#### Adding surrogate variables into the DESeq2 analysis
#################
#################

dds$SV1 <- svseq$sv[,1]
dds$SV2 <- svseq$sv[,2]
dds$SV3 <- svseq$sv[,3]
design(dds) <- ~ SV1 + SV2 + SV3 + race1 + mage + sex + gadays + zcat4 + medu + ASD


# Export variance stabilized counts, which will be influenced by the new design above
# vsd will be useful for plots and future figure generation
# CAUTION: THIS STEP TAKES SOME TIME! RUN TIME IS ABOUT 20 MINUTES FOR MRNASEQ DATA ON JULIA'S COMPUTER
vsd <- varianceStabilizingTransformation(dds, blind=FALSE)
vsd_matrix <-as.matrix(assay(vsd))
write.csv(vsd_matrix, paste0(Output_Folder,"/", cur_date, "_VSDCounts_IncludedSubjects.csv"), row.names=TRUE)



#################
#################
#### Evaluating variance stabilized values to guage whether SVs accounted for unwanted sources of variation between samples
#################
#################

plotPCA(vsd, intgroup="sex", returnData=FALSE)
plotPCA(vsd, intgroup="mage", returnData=FALSE)
plotPCA(vsd, intgroup="SV1", returnData=FALSE)
#SV1 accounts for a lot of variation along PC1 as shown by the steady gradient in color across the PC1 axis
plotPCA(vsd, intgroup="SV2", returnData=FALSE)
plotPCA(vsd, intgroup="SV3", returnData=FALSE)
#SV2 and 3 also account for a lot of variation across PCs

#In sum, it looks like the SVs are accounting for a lot of the unwanted variation which is great!



#################################################################################################
#################################################################################################
#### Statistical analysis to detect significantly differentially expressed genes
#################################################################################################
#################################################################################################

# Running the differential expression statistical pipeline
# CAUTION: THIS STEP TAKES A LOT OF TIME! RUN TIME IS ABOUT 1 HOUR FOR MRNASEQ DATA ON JULIA'S COMPUTER
dds <- DESeq(dds, betaPrior=FALSE)      # because we used a user-defined model matrix, need to set betaPrior=FALSE
# This ran a Wald test p-value (ASD 1 vs. 0)

resultsNames(dds) #check the available comparisons

# Pulling statistical results
res <- results(dds, pAdjustMethod = "BH")  #The  with multiple test correction by the default, BH (aka FDR)
head(res)


# Exporting statistical results:
write.csv(as.data.frame(res)[order(res$padj),], paste0(Output_Folder,"/", cur_date,"_AllStatResults.csv"))


#################################################################################################
#################################################################################################
#### Overview of statistical results
#################################################################################################
#################################################################################################
summary(res)

# Basic filters
table(res$padj<0.10)

table(res$padj<0.10 & res$log2FoldChange > log2(1.5))
table(res$padj<0.10 & res$log2FoldChange < -log2(1.5))

table(res$padj<0.10 & res$log2FoldChange > log2(1.3))
table(res$padj<0.10 & res$log2FoldChange < -log2(1.3))

table(res$padj<0.10 & (res$log2FoldChange > log2(1.5) | res$log2FoldChange < -log2(1.5)))


# Sort results and create a dataframe (df) for exporting & certain plots in the code below
resdf <- as.data.frame(res)[order(res$padj),]



# Also create a df for filtered data for certain plots in the code below
resdf_filter <- resdf[which(resdf$padj<0.10 & (resdf$log2FoldChange > log2(1.5) | resdf$log2FoldChange < -log2(1.5))),]
resdf_pfilter <- resdf[which(resdf$padj<0.10),]

Plotting

In [ ]:
###Heatmap###

#make dataset for heatmap in partek
countsforhm <- vsd_matrix

resdf_pfilter <- resdf_pfilter %>% 
  rownames_to_column("gene") %>% 
  filter(abs(log2FoldChange)>log2(1.3)) %>% 
  arrange(log2FoldChange)

genenames <- as.data.frame(resdf_pfilter$gene) %>% 
  rename("gene"="resdf_pfilter$gene")

countsforhm <- as.data.frame(countsforhm) %>% 
  rownames_to_column(var="gene") 

countsforhm <- left_join(genenames, countsforhm, by="gene") %>% 
  column_to_rownames(var="gene")

countsforhm <- as.data.frame(t(countsforhm)) %>% 
  rownames_to_column(var="ID") 

countsforhm <- full_join(countsforhm, coldata, by="ID") %>% 
  select(-"race",-"gadays",- "mage")

countsforhm <- countsforhm %>% 
  column_to_rownames(var="ID")

write.csv(countsforhm, paste0(Output_Folder,"/", cur_date,"_countdata_forheatmap_mrnas.csv"))

### MA plot ###

###################################
####  MA plot in ggplot2
###################################
MA <- resdf
MA[is.na(MA)] <- 1
MA1 <- MA[ which(MA$padj>=0.1),]  # all the rest
MA2 <- MA[ which(MA$padj<0.1 & MA$log2FoldChange > log2(1.3)),]   # sig up-regulated w/ FC filter-FIREBRICK
MA3 <- MA[ which(MA$padj<0.1 & MA$log2FoldChange < -log2(1.3)),]  # sig down-regulated w/ FC filter-dodgerblue4
MA4 <- MA[ which(MA$padj<0.1 & MA$log2FoldChange > 0 & MA$log2FoldChange <= log2(1.3)),]  # sig up-regulated w/o FC filter- darkorchid3
MA5 <- MA[ which(MA$padj<0.1 & MA$log2FoldChange < 0 & MA$log2FoldChange >= -log2(1.3)), ] # sig down-regulated w/o FC filter- chartreuse4

# get names of probes if annotating the graphic with them, according to specific criteria (not used here):
namesmmu=rownames(MA)[which((MA$log2FoldChange > 2 | MA$log2FoldChange < -2) & MA$padj < 0.1 & MA$baseMean > 0)]
namesmmu2=rownames(MA2)[which((MA2$log2FoldChange > 2 | MA2$log2FoldChange < -2) & MA2$padj < 0.1 & MA2$padj >= 0.05 & MA2$baseMean > 0)]
namesmmu3=rownames(MA3)[which((MA3$log2FoldChange > 2 | MA3$log2FoldChange < -2) & MA3$padj < 0.05 & MA3$baseMean > 0)]                                               

plot <-
  ggplot(subset(MA1, baseMean>=0), aes(x = baseMean, y = log2FoldChange)) + #can subset to exclude the very low counts (<1 average count) if we want
  geom_point(color="gray56", size = 2) + 
  geom_point(data = MA2, color="firebrick", size=3, show.legend = TRUE) + # colors used: firebrick, dodgerblue4, chocolate1, darkorchid3, chartreuse4
  geom_point(data = MA3, color="dodgerblue4", size=3, show.legend = TRUE) +
  geom_point(data = MA4, color="darkorchid3", size=3, show.legend = TRUE) +
  geom_point(data = MA5, color="chartreuse4", size=3, show.legend = TRUE) +
  theme_bw() +
  scale_x_continuous(trans = "log10", breaks=c(1,10,100, 1000, 10000, 100000, 1000000), labels=c("1","10","100", "1000", "10000", "100000", "1000000")) + 
  scale_y_continuous(limits=c(-1.5, 1.5), breaks=c(1.5,1,0.5,0,-0.5,-1,-1.5), labels=c(1.5,1,0.5,0,-0.5,-1,-1.5)) +
  xlab("Expression (Normalized Count)") + ylab("Fold Change (log2) Cases/Control") +
  labs(title="ASD vs. Control") +   
  #coord_fixed(ratio=0.5) +
  geom_hline(yintercept=0) 
#annotate("text",label="# DEPs higher in Cr(VI): 78\n(representing 77 DEGs)", x=1.1,y=10, size=4, hjust=0, color="darkorchid3") +
#annotate("text",label="# DEPs higher in Captan: 86\n(representing 85 DEGs)", x=1.1,y=-10, size=4, hjust=0, color="chartreuse4")+
#geom_text(data=MA2[namesmmu2,], size=3, hjust=-.05, vjust=.7, col="deepskyblue", aes(label=namesmmu2,size=5, angle=0)) + #annotate specific probes with text
#geom_text(data=MA3[namesmmu3,], size=3, hjust=-.05, vjust=.7, col="firebrick3", aes(label=namesmmu3,size=5, angle=0))

plot

#export single plots
png(paste0(Output_Folder, "/", cur_date, "_MAplot_v1.png"),
    width = 1600, height = 1200, units = "px", res = 250)
plot
dev.off()